# SETTING ENVIRONMENT


In [27]:
"""# mount the colab with google drive
from google.colab import drive
drive.mount('/content/drive')"""

"# mount the colab with google drive\nfrom google.colab import drive\ndrive.mount('/content/drive')"

In [28]:
# set folder tempat kerja (current working directory)
import os
cwd = '/Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita'
# cwd = '/content/drive/MyDrive/Monitoring Berita'

os.chdir(cwd)

# INPUT PARAMETER PENCARIAN

In [70]:
# AI Model (deepseek/openai)
model_name          = "openai"
# Tanggal awal pencarian berita
start_date          = "2025-09-26"
# Tanggal akhir pencarian berita
end_date            = "2025-09-26"
# Keywords pencarian berita
keywords_pencarian  = ["purbaya", "mbg", "stimulus"]
# Keywords analisis topic
keywords_topic      = ["bantuan sosial"]
# Jumlah halaman pencarian
page_length         = 5

# LANGKAH ANALISIS

## SIMPAN PARAMETER PENCARIAN

In [71]:
# tandai waktu mulai proses
import time 
start_process_time = time.time()

In [72]:
from datetime import datetime, timedelta

# Konversi string ke datetime
start = datetime.strptime(start_date, "%Y-%m-%d")
end = datetime.strptime(end_date, "%Y-%m-%d")

# Buat list tanggal dari end_date ke start_date (descending)
search_date = [(end - timedelta(days=i)).strftime("%Y-%m-%d")
               for i in range((end - start).days + 1)]

In [73]:
import json
from pathlib import Path

config_path = Path("config.json")

# Data baru yang ingin disimpan/diupdate
variabel = {
    "search_date": search_date,
    "keywords": keywords_pencarian,
    "topic_keywords":keywords_topic,
    "max_page_length": page_length,
    "AI_name": model_name
}

# Baca isi config.json lama (jika ada)
if config_path.exists():
    try:
        with open(config_path, "r", encoding="utf-8") as f:
            config = json.load(f)
    except Exception as e:
        print(f"Gagal membaca config.json: {e}")
        config = {}
else:
    config = {}

# Update hanya key yang ada di `variabel`
config.update(variabel)

# Simpan kembali ke file
with open(config_path, "w", encoding="utf-8") as f:
    json.dump(config, f, indent=4, ensure_ascii=False)

print(f"Config.json berhasil diperbarui: {config_path}")


Config.json berhasil diperbarui: config.json


## SCRAP DATA BERITA DI MEDIA ONLINE

### detik.com

In [33]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_detik_search_method.ipynb')

%run "$file_loc"

In [34]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_detik.ipynb')

%run "$file_loc"

15:09:27 | INFO | Proses tanggal 2025-09-26 | Halaman 1 | URL: https://news.detik.com/berita/indeks?page=1&date=09/26/2025
15:09:28 | INFO |   Ditemukan 168 item artikel pada halaman ini.
15:09:28 | INFO |   Baru: 141 | Duplikat: 26 | Total unik 141.
15:09:29 | INFO | Proses tanggal 2025-09-26 | Halaman 2 | URL: https://news.detik.com/berita/indeks?page=2&date=09/26/2025
15:09:29 | INFO |   Ditemukan 168 item artikel pada halaman ini.
15:09:29 | INFO |   Baru: 0 | Duplikat: 167 | Total unik 141.
15:09:29 | INFO |   Tidak ada URL baru pada halaman ini (konten berulang). Stop iterasi tanggal ini.


,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,Wagub Lampung Apresiasi Aksi Sosial Peserta Nomadic 4x4 Adventure Awaits,14 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8131721/wagub-lampung-apresiasi-aksi-sosial-peserta-nomadic-4x4-adventure-awaits,False,
1,"Rumah di Kompleks Kodamar Kelapa Gading Kebakaran, Api Masih Menyala",15 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8131720/rumah-di-kompleks-kodamar-kelapa-gading-kebakaran-api-masih-menyala,False,
2,Kandang-Rumah Potong Hewan di Depok Terancam Ditutup Buntut Cemari 2 Setu,17 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8131718/kandang-rumah-potong-hewan-di-depok-terancam-ditutup-buntut-cemari-2-setu,False,
3,Pramono Cerita Atasi Polemik Kampung Bayam: Alhamdulillah Tak Ada Lagi Protes,20 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8131714/pramono-cerita-atasi-polemik-kampung-bayam-alhamdulillah-tak-ada-lagi-protes,False,
4,"Arahan Dukung Prabowo-Gibran 2 Periode Disorot, Jokowi Buka Suara",20 menit yang lalu,Tidak Diketahui,https://news.detik.com/berita/d-8131713/arahan-dukung-prabowo-gibran-2-periode-disorot-jokowi-buka-suara,False,
...,...,...,...,...,...,...
136,Disclaimer,2025-09-26,Tidak Diketahui,https://www.detik.com/disclaimer,False,
137,Insertlive,2025-09-26,Tidak Diketahui,https://www.insertlive.com/,False,
138,Beautynesia,2025-09-26,Tidak Diketahui,https://beautynesia.id,False,
139,Female Daily,2025-09-26,Tidak Diketahui,https://femaledaily.com,False,



--- Ringkasan ---
Total artikel: 141
Artikel relevan: 2
5 contoh judul relevan:
1. Food Tray MBG Wajib SNI Tahun Ini, Ompreng Tak Standar Dilarang Edar
2. Wamensesneg Yakin Prabowo Sudah Tahu Kasus Ribuan Siswa Keracunan MBG


### cnbcindoncesia.com

In [35]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_cnbc.ipynb')

%run "$file_loc"

15:09:29 | INFO | Inisialisasi selesai.
15:09:29 | INFO | Memproses tanggal: 2025-09-26
15:09:29 | INFO | [2025-09-26] Page 1 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=1 | Artikel ditemukan (kandidat): 10
15:09:29 | INFO | [2025-09-26] Page 1 | Artikel cocok tanggal 2025-09-26: 10
15:09:30 | INFO | [2025-09-26] Page 2 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=2 | Artikel ditemukan (kandidat): 10
15:09:30 | INFO | [2025-09-26] Page 2 | Artikel cocok tanggal 2025-09-26: 10
15:09:31 | INFO | [2025-09-26] Page 3 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=3 | Artikel ditemukan (kandidat): 10
15:09:31 | INFO | [2025-09-26] Page 3 | Artikel cocok tanggal 2025-09-26: 10
15:09:32 | INFO | [2025-09-26] Page 4 | URL: https://www.cnbcindonesia.com/indeks?tipe=artikel&page=4 | Artikel ditemukan (kandidat): 10
15:09:32 | INFO | [2025-09-26] Page 4 | Artikel cocok tanggal 2025-09-26: 10
15:09:34 | INFO | [2025-09-26] Page 5 | URL: https://www

,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,Tambahan Pasokan BBM Untuk SPBU Swasta Shell-BP Sudah Tiba di RINews3 menit yang lalu,2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926145959-4-670514/tambahan-pasokan-bbm-untuk-spbu-swasta-shell-bp-sudah-tiba-di-ri,False,
1,"Adrian Gunadi Sering Viral di Medsos, Berkeliaran di Doha Walau BuronTech4 menit yang lalu",2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/tech/20250926131336-37-670478/adrian-gunadi-sering-viral-di-medsos-berkeliaran-di-doha-walau-buron,False,
2,"InternasionalTrump Puja-puji Erdogan, Sebut Tegas & Berpendirian, Kok Bisa?News9 menit yang lalu",2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926140442-4-670498/trump-puja-puji-erdogan-sebut-tegas-berpendirian-kok-bisa,False,
3,Begini Manuver Purbaya Bersihkan RI dari Rokok IlegalNews14 menit yang lalu,2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926145215-4-670511/begini-manuver-purbaya-bersihkan-ri-dari-rokok-ilegal,True,purbaya
4,"RI Nambah Lebih 10% Saham di Freeport, BUMD Papua Akan Kebagian JatahNews17 menit yang lalu",2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926144408-4-670508/ri-nambah-lebih-10-saham-di-freeport-bumd-papua-akan-kebagian-jatah,False,
5,"Purbaya Pastikan SMI, PII Hingga LPEI Tetap di Bawah KemenkeuMarket19 menit yang lalu",2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/market/20250926143925-17-670504/purbaya-pastikan-smi-pii-hingga-lpei-tetap-di-bawah-kemenkeu,True,purbaya
6,"Harta Haji Isam Tembus Rp 100 T, Lewati Bos Alfamart & Bos NikelMarket21 menit yang lalu",2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/market/20250926141323-17-670499/harta-haji-isam-tembus-rp-100-t-lewati-bos-alfamart-bos-nikel,False,
7,"Meningkat 8,03%, Ekspor RI Sentuh US$ 160 M hingga Juli 2025News22 menit yang lalu",2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926144042-4-670505/meningkat-803-ekspor-ri-sentuh-us--160-m-hingga-juli-2025,False,
8,"Revisi UU BUMN Hanya Larang Menteri & Wamen Rangkap Jabatan, Eselon?Market24 menit yang lalu",2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/market/20250926132216-17-670471/revisi-uu-bumn-hanya-larang-menteri-wamen-rangkap-jabatan-eselon,False,
9,Breaking! Purbaya: Cukai Rokok 2026 Tidak NaikNews25 menit yang lalu,2025-09-26,Tidak Diketahui,https://www.cnbcindonesia.com/news/20250926144104-4-670506/breaking-purbaya-cukai-rokok-2026-tidak-naik,True,purbaya



--- Ringkasan ---
Total artikel: 50
Artikel relevan: 7
Contoh judul relevan:
1. Begini Manuver Purbaya Bersihkan RI dari Rokok IlegalNews14 menit yang lalu
2. Purbaya Pastikan SMI, PII Hingga LPEI Tetap di Bawah KemenkeuMarket19 menit yang lalu
3. Breaking! Purbaya: Cukai Rokok 2026 Tidak NaikNews25 menit yang lalu
4. Purbaya Makan Siang Bareng Bos BI, Ini yang Dibahas!Market50 menit yang lalu
5. Fakta Keracunan Massal Menu MBG, Ada Lauk Ikan Hiu & Respons BGNLifestyle1 jam yang lalu


### idttimes.com

In [36]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_idn_times_search_method.ipynb')

%run "$file_loc"

### kompas.com

In [37]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_kompas.ipynb')

%run "$file_loc"

15:10:22 | INFO | Environment ready. Libraries loaded.
15:10:22 | INFO | Memproses tanggal: 2025-09-26, site: all
15:10:22 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-26&page=1
15:10:23 | INFO | Kandidat link artikel di page 1: 40
15:10:23 | INFO | Artikel berhasil diparse dari page 1: 39
15:10:24 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-26&page=2
15:10:24 | INFO | Kandidat link artikel di page 2: 40
15:10:24 | INFO | Artikel berhasil diparse dari page 2: 39
15:10:25 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-26&page=3
15:10:25 | INFO | Kandidat link artikel di page 3: 41
15:10:26 | INFO | Artikel berhasil diparse dari page 3: 40
15:10:27 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-26&page=4
15:10:27 | INFO | Kandidat link artikel di page 4: 41
15:10:27 | INFO | Artikel berhasil diparse dari page 4: 40
15:10:28 | INFO | Fetch: https://indeks.kompas.com/?site=all&date=2025-09-26&page=5
15:10:28 | INF

,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,"Konser Musik di Makassar Ricuh, Penonton Tanpa Tiket Jebol Pagar, 4 Orang LukaREGIONAL26/09/2025",2025-09-26,Tidak Diketahui,https://makassar.kompas.com/read/2025/09/26/150734678/konser-musik-di-makassar-ricuh-penonton-tanpa-tiket-jebol-pagar-4-orang,False,
1,"Pertemuan Hangat Jokowi dam Hasan Nasbi, Nostalgia Hingga Pesan untuk PertaminaREGIONAL26/09/2025",2025-09-26,Tidak Diketahui,https://regional.kompas.com/read/2025/09/26/150638378/pertemuan-hangat-jokowi-dam-hasan-nasbi-nostalgia-hingga-pesan-untuk,False,
2,Pemberi Layanan Dinilai Pihak Paling Bertanggung Jawab atas Keracunan MBGREGIONAL26/09/2025,2025-09-26,Tidak Diketahui,https://regional.kompas.com/read/2025/09/26/150609278/pemberi-layanan-dinilai-pihak-paling-bertanggung-jawab-atas-keracunan-mbg,False,
3,Saran Kemenaker Buat Gen Z yang Cari KerjaMONEY26/09/2025,2025-09-26,Tidak Diketahui,https://money.kompas.com/read/2025/09/26/150400226/saran-kemenaker-buat-gen-z-yang-cari-kerja-,False,
4,"Polisi Gerebek Gudang Oplosan Gas Subsidi, Diduga Dibekingi AparatREGIONAL26/09/2025",2025-09-26,Tidak Diketahui,https://medan.kompas.com/read/2025/09/26/150335278/polisi-gerebek-gudang-oplosan-gas-subsidi-diduga-dibekingi-aparat,False,
...,...,...,...,...,...,...
192,"Larangan Diabaikan, Motor Masih Ramai Melintas di Flyover Pesing meski Rawan KecelakaanNEWS26/09/2025",2025-09-26,Tidak Diketahui,https://megapolitan.kompas.com/read/2025/09/26/12305261/larangan-diabaikan-motor-masih-ramai-melintas-di-flyover-pesing-meski,False,
193,Peneliti Ciptakan Obat Baru untuk Turunkan Berat Badan Guna Cegah ObesitasTREN26/09/2025,2025-09-26,Tidak Diketahui,https://www.kompas.com/tren/read/2025/09/26/123000165/peneliti-ciptakan-obat-baru-untuk-turunkan-berat-badan-guna-cegah-obesitas,False,
194,Siapa Brigjen Pol Djuhandhani Rahardjo Puro Kapolda Sulsel Baru yang Pernah Tangani Ijazah Jokowi?PROV26/09/2025,2025-09-26,Tidak Diketahui,https://www.kompas.com/sulawesi-selatan/read/2025/09/26/123000688/siapa-brigjen-pol-djuhandhani-rahardjo-puro-kapolda-sulsel-baru,False,
195,"Profil Brigjen Helfi Assegaf: Ahli Reserse, Bongkar Kasus Beras Oplosan, Jadi Kapolda LampungPROV26/09/2025",2025-09-26,Tidak Diketahui,https://www.kompas.com/jawa-barat/read/2025/09/26/123000088/profil-brigjen-helfi-assegaf--ahli-reserse-bongkar-kasus-beras,False,



--- Ringkasan ---
Total artikel: 197
Artikel relevan: 16
Contoh 5 judul relevan:
1. Labkesda Jabar Ungkap Bakteri Penyebab Keracunan MBG, dari Salmonella hingga E-ColiPROV26/09/2025
2. Cegah Keracunan MBG Terulang, SPPG Sumedang Ditertibkan dan DitambahREGIONAL26/09/2025
3. Cucun Ahmad Semprot Kepala SPPG Usai 411 Siswa Keracunan MBG: Sudah Langgar SOPPROV26/09/2025
4. 13 Siswa Mual Muntah, MBG di SDN 178 Palembang Dihentikan SementaraREGIONAL26/09/2025
5. Keracunan Massal Terus Terjadi, Program MBG Dinilai Minim Pengawasan dan KajianREGIONAL26/09/2025


### liputan6.com

In [38]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_liputan6.ipynb')

%run "$file_loc"

15:10:30 | INFO | Memproses tanggal [News]: 2025-09-26


15:10:30 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/26?page=1
15:10:30 | INFO | News halaman 1: ditemukan 27 artikel
15:10:31 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/26?page=2
15:10:31 | INFO | News halaman 2: ditemukan 18 artikel
15:10:32 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/26?page=3
15:10:32 | INFO | News halaman 3: ditemukan 4 artikel
15:10:33 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/26?page=4
15:10:34 | INFO | News halaman 4: ditemukan 4 artikel
15:10:35 | INFO | GET News: https://www.liputan6.com/news/indeks/2025/09/26?page=5
15:10:35 | INFO | News halaman 5: ditemukan 4 artikel
15:10:36 | INFO | Memproses tanggal [Bisnis]: 2025-09-26
15:10:36 | INFO | GET Bisnis: https://www.liputan6.com/bisnis/indeks/2025/09/26?page=1
15:10:37 | INFO | Bisnis halaman 1: ditemukan 24 artikel
15:10:38 | INFO | GET Bisnis: https://www.liputan6.com/bisnis/indeks/2025/09/26?page=2
15:10:38 | INFO | Bisnis 

,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,Cek Fakta: Hoaks Artikel Cak Imin Dapat Sedikit Hasil Korupsi Kuota Haji dari Yaqut Cholil Qoumas,2025-09-26T13:00:00+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/6169206/cek-fakta-hoaks-artikel-cak-imin-dapat-sedikit-hasil-korupsi-kuota-haji-dari-yaqut-cholil-qoumas,False,
1,"Pertamina Patra Niaga Imbau Masyarakat Waspada Hoaks yang Beredar, Simak Faktanya",2025-09-26T12:54:04+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/6169285/pertamina-patra-niaga-imbau-masyarakat-waspada-hoaks-yang-beredar-simak-faktanya,False,
2,Cek Fakta: Tidak Benar Dalam Video Ini Pom Bensin Dibakar Massa karena Motor Pajak Mati Tak Boleh Isi BBM,2025-09-26T11:00:35+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/6168870/cek-fakta-tidak-benar-dalam-video-ini-pom-bensin-dibakar-massa-karena-motor-pajak-mati-tak-boleh-isi-bbm,False,
3,"Deretan Hoaks Seputar Yaqut Cholil Qoumas yang Beredar di Publik, Simak Daftarnya",2025-09-26T09:00:00+07:00,Tidak Diketahui,https://www.liputan6.com/cek-fakta/read/6168525/deretan-hoaks-seputar-yaqut-cholil-qoumas-yang-beredar-di-publik-simak-daftarnya,False,
4,"Kejagung Geledah Kantor PT SEI, Cari Bukti Kasus Dugaan Korupsi Akuisisi Blok Migas",2025-09-26T14:57:26+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6169456/kejagung-geledah-kantor-pt-sei-cari-bukti-kasus-dugaan-korupsi-akuisisi-blok-migas,False,
5,Mentan Amran Dorong Mahasiswa Jadi Penggerak Pertanian Masa Depan,2025-09-26T14:50:05+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6169453/mentan-amran-dorong-mahasiswa-jadi-penggerak-pertanian-masa-depan,False,
6,09:44,2025-09-26T14:38:31+07:00,Tidak Diketahui,https://enamplus.liputan6.com/news/read/6169417/kejutan-respons-menko-ahy-soal-viral-macet-jakarta-tegas-perintahkan-ini,False,
7,"Profil dr Tan Shot Yen, Ahli Gizi Lulusan Filsafat yang Kritik Tajam MBG di Rapat DPR",2025-09-26T14:37:19+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6169437/profil-dr-tan-shot-yen-ahli-gizi-lulusan-filsafat-yang-kritik-tajam-mbg-di-rapat-dpr,True,mbg
8,"Komplotan Begal Bersenjata Bikin Ulah di Cakung, Warga Mau ke Pasar Dibacok Lalu Motor Dir...",2025-09-26T14:17:33+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6169397/komplotan-begal-bersenjata-bikin-ulah-di-cakung-warga-mau-ke-pasar-dibacok-lalu-motor-dirampas,False,
9,Indonesia Halal Industry Award 2025: Kaltim Raih Penghargaan Lembaga Pemerintahan Pendukun...,2025-09-26T13:29:53+07:00,Tidak Diketahui,https://www.liputan6.com/news/read/6169349/indonesia-halal-industry-award-2025-kaltim-raih-penghargaan-lembaga-pemerintahan-pendukung-program-halal-terbaik-di,False,



==== RINGKASAN ====
Total artikel: 60
Artikel relevan: 7
Contoh 5 judul relevan:
1. Profil dr Tan Shot Yen, Ahli Gizi Lulusan Filsafat yang Kritik Tajam MBG di Rapat DPR
2. Penjelasan BGN Soal Keracunan Massal Siswa di Ketapang usai Santap Menu MBG Berbahan Ikan ...
3. Top 3 News: Istana Tanggapi Usulan Setop Sementara MBG Imbas Banyak Kasus Keracunan
4. Banyak Siswa Keracunan MBG, Cucun DPR: Jangan Program Visioner Ini Rusak Karena Lemahnya P...
5. Solusi Agar MBG Tak Makan Korban Lagi


### cnnindonesia.com

In [39]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_cnn.ipynb')

%run "$file_loc"

15:10:43 | INFO | Project root: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita
15:10:43 | INFO | Keywords: ['purbaya', 'mbg', 'stimulus']
15:10:43 | INFO | Using TAG mode for: purbaya
15:10:49 | INFO | [tag:purbaya] page 1 -> 36 items
15:10:56 | INFO | [tag:purbaya] page 2 -> 34 items
15:11:01 | INFO | [tag:purbaya] page 3 -> 28 items
15:11:02 | INFO | Using TAG mode for: mbg
15:11:07 | INFO | [tag:mbg] page 1 -> 36 items
15:11:13 | INFO | [tag:mbg] page 2 -> 34 items
15:11:18 | INFO | [tag:mbg] page 3 -> 34 items
15:11:19 | INFO | Using TAG mode for: stimulus
15:11:23 | INFO | [tag:stimulus] page 1 -> 36 items
15:11:28 | INFO | [tag:stimulus] page 2 -> 34 items
15:11:36 | INFO | [tag:stimulus] page 3 -> 34 items
15:11:37 | INFO | Total unique links: 114
15:11:37 | INFO | Saved: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/daftar_berita/cnn_links.csv


,judul_berita,url_berita,keyword,page,published_at,scraped_at
0,Keluar,https://connect.detik.com/oauth/signout?redirectUrl=https%3A%2F%2Fwww.cnnindonesia.com%2Ftag%2Fpurbaya,purbaya,1,NaT,2025-09-26 15:10:44.685841+07:00
1,DAFTAR,https://connect.detik.com/accounts/register?clientId=10027&redirectUrl=https%3A%2F%2Fwww.cnnindonesia.com%2Fauthorize&backURL=https%3A%2F%2Fwww.cnnindonesia.com%2Ftag%2Fpurbaya&ui=apps&osType=ANDROID,purbaya,1,NaT,2025-09-26 15:10:44.757844+07:00
2,Politik,https://www.cnnindonesia.com/nasional/politik,purbaya,1,NaT,2025-09-26 15:10:44.915460+07:00
3,Hukum & Kriminal,https://www.cnnindonesia.com/nasional/hukum-kriminal,purbaya,1,NaT,2025-09-26 15:10:45.119888+07:00
4,Peristiwa,https://www.cnnindonesia.com/nasional/peristiwa,purbaya,1,NaT,2025-09-26 15:10:45.237239+07:00
5,Pemilu,https://www.cnnindonesia.com/nasional/pemilu,purbaya,1,NaT,2025-09-26 15:10:45.346652+07:00
6,Info Politik,https://www.cnnindonesia.com/nasional/info-politik,purbaya,1,NaT,2025-09-26 15:10:45.454339+07:00
7,Asia Pasifik,https://www.cnnindonesia.com/internasional/asia-pasifik,purbaya,1,NaT,2025-09-26 15:10:45.623439+07:00
8,Timur Tengah,https://www.cnnindonesia.com/internasional/timur-tengah,purbaya,1,NaT,2025-09-26 15:10:45.695446+07:00
9,Eropa Amerika,https://www.cnnindonesia.com/internasional/eropa-amerika,purbaya,1,NaT,2025-09-26 15:10:45.808443+07:00


Output Excel: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/daftar_berita/cnn_links.csv


### okezone.com

In [40]:
file_loc = os.path.join(cwd, 'tarik_daftar_berita', 'list_berita_okezone.ipynb')

%run "$file_loc"

15:11:37 | WARNING | Config tidak ditemukan di /content/drive/MyDrive/Monitoring Berita/config.json. Menggunakan default.
15:11:37 | INFO | topic_keywords: ['ekonomi', 'bursa', 'saham', 'investasi', 'keuangan', 'bank', 'rupiah', 'inflasi', 'bisnis']
15:11:37 | INFO | dates: ['2025-09-26']
15:11:37 | INFO | max_pages_per_date: 5
15:11:37 | INFO | Memproses tanggal: 2025-09-26
15:11:40 | INFO | [2025-09-26] URL: https://index.okezone.com/bydate/index/2025/09/26/0/ | Artikel ditemukan: 15
15:11:40 | INFO | Artikel diparsing (page 1): 15
15:11:51 | WARNING | Timeout https://index.okezone.com/home/index/15/ (try 1/3)
15:12:02 | WARNING | Timeout https://index.okezone.com/home/index/15/ (try 2/3)
15:12:14 | WARNING | Timeout https://index.okezone.com/home/index/15/ (try 3/3)
15:12:18 | WARNING | Gagal memuat / tidak ada konten: https://index.okezone.com/home/index/15/. Stop tanggal ini.


,judul_berita,tanggal_berita,penulis_berita,url_berita,relevan,keywords_found
0,Hasil Perempatfinal Korea Open 2025: Alwi Farhan Tantang Jonatan Christie di 4 Besar!,"Netting | Jum'at, 26 September 2025 15:01 WIB Hasil Perempatfinal Korea Open 2025: Alwi Farhan Tantang Jonatan Christie di 4 Besar!",Tidak Diketahui,https://sports.okezone.com/read/2025/09/26/40/3172646/hasil-perempatfinal-korea-open-2025-alwi-farhan-tantang-jonatan-christie-di-4-besar,False,
1,"Tertinggal di Revolusi Industri, Ekonomi RI Diprediksi Tumbuh 10 Persen Berkat AI","Hot Issue | Jum'at, 26 September 2025 14:55 WIB Tertinggal di Revolusi Industri, Ekonomi RI Diprediksi Tumbuh 10 Persen Berkat AI",Tidak Diketahui,https://economy.okezone.com/read/2025/09/26/320/3172645/tertinggal-di-revolusi-industri-ekonomi-ri-diprediksi-tumbuh-10-persen-berkat-ai,True,ekonomi
2,Ini Sebabnya Sule Ditilang Petugas Dishub Jakarta,"Hot Gossip | Jum'at, 26 September 2025 14:53 WIB Ini Sebabnya Sule Ditilang Petugas Dishub Jakarta",Tidak Diketahui,https://celebrity.okezone.com/read/2025/09/26/33/3172644/ini-sebabnya-sule-ditilang-petugas-dishub-jakarta,False,
3,"Bayern Munich Tampil Gahar di Awal Musim 2025-2026, Vincent Kompany Bocorkan Rahasianya","Sepakbola Dunia | Jum'at, 26 September 2025 14:52 WIB Bayern Munich Tampil Gahar di Awal Musim 2025-2026, Vincent Kompany Bocorkan Rahasianya",Tidak Diketahui,https://bola.okezone.com/read/2025/09/26/51/3172643/bayern-munich-tampil-gahar-di-awal-musim-2025-2026-vincent-kompany-bocorkan-rahasianya,False,
4,"Jadwal dan Link Live Streaming Vision+ Borussia Monchengladbach vs Eintracht Frankfurt di Bundesliga 2025-2026, Bisa Nonton Gratis!","Sepakbola Dunia | Jum'at, 26 September 2025 14:49 WIB Jadwal dan Link Live Streaming Vision+ Borussia Monchengladbach vs Eintracht Frankfurt di Bundesliga 2025-2026, Bisa Nonton Gratis!",Tidak Diketahui,https://bola.okezone.com/read/2025/09/26/51/3172642/jadwal-dan-link-live-streaming-vision-borussia-monchengladbach-vs-eintracht-frankfurt-di-bundesliga-2025-2026-bisa-nonton-gratis,False,
5,"Momen Bersejarah, Wanita Tertinggi dan Terpendek di Dunia Saling Bertemu","Life | Jum'at, 26 September 2025 14:49 WIB Momen Bersejarah, Wanita Tertinggi dan Terpendek di Dunia Saling Bertemu",Tidak Diketahui,https://women.okezone.com/read/2025/09/26/612/3172641/momen-bersejarah-wanita-tertinggi-dan-terpendek-di-dunia-saling-bertemu,False,
6,"Kisah Cristiano Ronaldo yang Dituduh Kencani PSK Bertarif Rp156 Juta per Jam, Apa Respons CR7?","Sepakbola Dunia | Jum'at, 26 September 2025 14:49 WIB Kisah Cristiano Ronaldo yang Dituduh Kencani PSK Bertarif Rp156 Juta per Jam, Apa Respons CR7?",Tidak Diketahui,https://bola.okezone.com/read/2025/09/26/51/3172639/kisah-cristiano-ronaldo-yang-dituduh-kencani-psk-bertarif-rp156-juta-per-jam-apa-respons-cr7,False,
7,Microsoft Putus Akses Layanan untuk Militer Israel Terkait Pengawasan Warga Sipil Gaza,"Techno | Jum'at, 26 September 2025 14:48 WIB Microsoft Putus Akses Layanan untuk Militer Israel Terkait Pengawasan Warga Sipil Gaza",Tidak Diketahui,https://ototekno.okezone.com/read/2025/09/26/16/3172640/microsoft-putus-akses-layanan-untuk-militer-israel-terkait-pengawasan-warga-sipil-gaza,False,
8,"Panel Surya Ratusan Sekolah Pakistan Dicuri, Ribuan Siswa Alami Krisis Listrik","International | Jum'at, 26 September 2025 14:41 WIB Panel Surya Ratusan Sekolah Pakistan Dicuri, Ribuan Siswa Alami Krisis Listrik",Tidak Diketahui,https://news.okezone.com/read/2025/09/26/18/3172638/panel-surya-ratusan-sekolah-pakistan-dicuri-ribuan-siswa-alami-krisis-listrik,False,
9,"RUU BUMN Siap Disahkan di Paripurna, BP BUMN Gantikan Peran Kementerian BUMN","Hot Issue | Jum'at, 26 September 2025 14:41 WIB RUU BUMN Siap Disahkan di Paripurna, BP BUMN Gantikan Peran Kementerian BUMN",Tidak Diketahui,https://economy.okezone.com/read/2025/09/26/320/3172637/ruu-bumn-siap-disahkan-di-paripurna-bp-bumn-gantikan-peran-kementerian-bumn,False,



--- Ringkasan ---
Total artikel: 15
Artikel relevan: 1

Contoh 5 judul relevan:
- Tertinggal di Revolusi Industri, Ekonomi RI Diprediksi Tumbuh 10 Persen Berkat AI  |  keywords: ekonomi  |  https://economy.okezone.com/read/2025/09/26/320/3172645/tertinggal-di-revolusi-industri-ekonomi-ri-diprediksi-tumbuh-10-persen-berkat-ai


## MENGGABUNGKAN DATA

### Konsolidasi Data Url Berita

In [41]:
file_loc = os.path.join(cwd, '01_konsolidasi_data_berita.ipynb')

%run "$file_loc"

Membaca Excel:   0%|          | 0/9 [00:00<?, ?file/s]

15:12:18 | INFO | Duplikat dihapus: 36 baris | Total akhir: 517
15:12:18 | INFO | Selesai gabung. Total baris: 517 | File terbaca: 9
15:12:19 | WARNING | Gagal parse index=351, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=359, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=360, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=361, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=362, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=363, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=364, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=365, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | WARNING | Gagal parse index=367, nilai='<NA>': String does not contain a date: <NA>
15:12:19 | W

File berhasil disimpan: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/konsolidasi_berita/hasil_gabungan_20250926_151219.csv


## BACA BERITA

### Membaca Data Url Berita

In [42]:
file_loc = os.path.join(cwd, '02_baca_link_berita.ipynb')

%run "$file_loc"

15:12:20 | INFO | Total URL untuk diproses (dibatasi 100): 296


Scraping artikel:   0%|          | 0/296 [00:00<?, ?it/s]

15:13:38 | INFO | 
Summary status:
status
ok_trafilatura      173
ok_site_specific    123
Name: count, dtype: int64
15:13:38 | INFO | Tersimpan: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/hasil_baca_berita/hasil_scraping_artikel_20250926_151338.csv
15:13:38 | INFO | Berhasil update config.json di /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/config.json


## LABELLING ISU


In [43]:
file_loc = os.path.join(cwd, '03_labelling_awal.ipynb')

%run "$file_loc"

Normalisasi tanggal_berita: total=296, sukses=289, gagal=7 (2.4%)
Filter tanggal: 2025-09-26 s.d 2025-09-26 (dari 289 -> 111)
Labeling selesai. Jumlah baris: 111


,kategori_isu,jumlah
0,Isu Lainnya,41
1,Isu Nasional,35
2,Isu Internasional,19
3,Isu Kemenkeu,16


Tersimpan: /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/hasil_labeling/hasil_labeling_20250926_151339.csv & /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/hasil_labeling/hasil_labeling_20250926_151339.xlsx
Berhasil update config.json di /Users/yusufpradana/Library/CloudStorage/OneDrive-Personal/Pekerjaan BMN/05. 2025/98_monitoring_berita/config.json


## ANALISIS DENGAN GEN AI

In [46]:
file_loc = os.path.join(cwd, '04_analisis_dengan_genAI.ipynb')

%run "$file_loc"

API keys - Deepseek: OK | OpenAI: OK


Analisis berita:   0%|          | 0/70 [00:00<?, ?berita/s]

15:22:24 | WARNING | Attempt 1 gagal: Tidak ditemukan blok JSON dalam respons.
15:22:58 | WARNING | Attempt 1 gagal: Tidak ditemukan blok JSON dalam respons.
15:23:16 | WARNING | Attempt 2 gagal: Tidak ditemukan blok JSON dalam respons.
15:23:36 | WARNING | Attempt 3 gagal: Tidak ditemukan blok JSON dalam respons.
15:24:34 | WARNING | Attempt 1 gagal: Tidak ditemukan blok JSON dalam respons.
15:24:52 | WARNING | Attempt 2 gagal: Tidak ditemukan blok JSON dalam respons.
15:25:10 | WARNING | Attempt 3 gagal: Tidak ditemukan blok JSON dalam respons.
15:25:31 | WARNING | Attempt 1 gagal: Tidak ditemukan blok JSON dalam respons.
15:25:48 | WARNING | Attempt 2 gagal: Tidak ditemukan blok JSON dalam respons.
15:26:06 | WARNING | Attempt 3 gagal: Tidak ditemukan blok JSON dalam respons.
15:26:27 | WARNING | Attempt 1 gagal: Tidak ditemukan blok JSON dalam respons.
15:26:44 | WARNING | Attempt 2 gagal: Tidak ditemukan blok JSON dalam respons.
15:27:04 | WARNING | Attempt 3 gagal: Tidak ditemuka

Pipeline selesai. Hasil disimpan di analisis_ai_20250926.csv


15:54:46 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
15:54:46 | WARNING | Attempt 1 gagal: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}
15:54:48 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
15:54:48 | WARNING | Attempt 2 gagal: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not supported with this model. Use 'max_completion_tokens' instead.", 'type': 'invalid_request_error', 'param': 'max_tokens', 'code': 'unsupported_parameter'}}
15:54:51 | INFO | HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 400 Bad Request"
15:54:51 | WARNING | Attempt 3 gagal: Error code: 400 - {'error': {'message': "Unsupported parameter: 'max_tokens' is not s

Hasil uji LLM (openai): {'topik': 'Kemenkeu', 'subtopik': '', 'sentimen': 'netral', 'alasan_sentimen': 'Berita informatif tanpa penilaian. (fallback error: Error code: 400 - {\'error\': {\'message\': "Unsupported parameter: \'max_tokens\' is not supported with this model. Use \'max_completion_tokens\' instead.", \'type\': \'invalid_request_error\', \'param\': \'max_tokens\', \'code\': \'unsupported_parameter\'}})', 'confidence': 0.6}


In [47]:
# Tandai waktu selesai proses
end_process_time = time.time()
elapsed_time = end_process_time - start_process_time
# jadikan menit & detik
elapsed_minutes = int(elapsed_time // 60)
elapsed_seconds = int(elapsed_time % 60)
print(f"Waktu proses: {elapsed_minutes} menit {elapsed_seconds} detik")

Waktu proses: 47 menit 52 detik
